In [ ]:
#importing libraries
!pip install -U datasets tiktoken tqdm numpy torch matplotlib huggingface_hub
import os
import math
import time
from contextlib import nullcontext
from dataclasses import dataclass, field
from tqdm.auto import tqdm
import json
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR
import tiktoken
from datasets import load_dataset
import matplotlib.pyplot as plt

In [ ]:
#tokenization implementation
from datasets import load_dataset
import numpy as np
from tqdm.auto import tqdm
import os
import tiktoken 
import math


enc = tiktoken.get_encoding("gpt2")
eot_token = enc.eot_token

In [ ]:
def processing_bpe(sample):
    text_data = sample.get('text', '')
    if not isinstance(text_data, str): text_data = ""
    ids = enc.encode_ordinary(text_data)
    ids.append(eot_token)
    out = {'ids': ids, 'len': len(ids)}
    return out
combined_data_dir = config.data_dir
os.makedirs(combined_data_dir, exist_ok=True)
train_filename = os.path.join(combined_data_dir, 'train_combined.bin')
val_filename = os.path.join(combined_data_dir, 'val_combined.bin')

In [ ]:
#data import and division
if os.path.exists(train_filename) and os.path.exists(val_filename):
    print(f"Combined .bin files already exist in {combined_data_dir}. Skipping processing.")

else:
    ts_df = load_dataset("roneneldan/TinyStories")
    bc_df = load_dataset("rojagtap/bookcorpus", split='train')
    
    #Train/Validation Split
    ts_split = ts_df['train'].train_test_split(test_size=0.01, seed=42)
    ts_train_df = ts_split['train']
    ts_val_df = ts_split['test']
    
    bc_split = bc_df.train_test_split(test_size=0.01, seed=42)
    bc_train_df = bc_split['train']
    bc_val_df = bc_split['test']

    dataset_splits = {
        'train': {'ts': ts_train_df, 'bc': bc_train_df},
        'validation': {'ts': ts_val_df, 'bc': bc_val_df}
    }

    for split in ['train', 'validation']:
        filename = train_filename if split == 'train' else val_filename
        
        if os.path.exists(filename):
            print(f"Deleting existing {filename} to rebuild...")
            os.remove(filename)